In [8]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import math

# Load the trained model
model = load_model('sign_language_new_model.h5')

# Initialize video capture and hand detector
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)

# Constants
offset = 20
imgSize = 64  # Match the target size used in training
labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

def preprocess_image(img):
    # Convert to grayscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Resize to match training size
    img = cv2.resize(img, (imgSize, imgSize))
    # Normalize
    img = img / 255.0
    # Reshape for model input
    img_final = np.expand_dims(img, axis=[0, -1])
    return img_final, img  # Return both the model input and the normalized image for display

while True:
    success, img = cap.read()
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]
        
        if imgCrop.size > 0:  # Check if crop is valid
            aspectRatio = h / w
            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize
            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hCal + hGap, :] = imgResize
            
            # Preprocess and predict
            # Preprocess and predict
            processed_img, normalized_img = preprocess_image(imgWhite)
            prediction = model.predict(processed_img, verbose=0)
            index = np.argmax(prediction[0])
            confidence = prediction[0][index]
            
            # Draw results
            cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                         (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
            cv2.putText(imgOutput, f"{labels[index]} ({confidence:.2f})", 
                       (x, y - 26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
            cv2.rectangle(imgOutput, (x-offset, y-offset),
                         (x + w+offset, y + h+offset), (255, 0, 255), 4)
            
            # Show cropped and processed images
            cv2.imshow("ImageCrop", imgCrop)
            cv2.imshow("ImageWhite", imgWhite)
            # Show only the final preprocessed image
            cv2.imshow("Preprocessed Image", normalized_img)
    
    cv2.imshow("Image", imgOutput)
    
    # Break loop on 'q' press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows() 